In [1]:
from ml_t6.preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

#### Prepare

In [3]:
data = Preprocessing(
    X_link="/Users/jonas/Documents/Master_Vorlesungen/Semester_02/Practical Data Science/Final/PDS_Final/data/linkedin-cvs-annotated.json",
    seniority_link  = "/Users/jonas/Documents/Master_Vorlesungen/Semester_02/Practical Data Science/Final/PDS_Final/data/seniority-v2.csv",
    department_link = "/Users/jonas/Documents/Master_Vorlesungen/Semester_02/Practical Data Science/Final/PDS_Final/data/department-v2.csv"
)

data.data_pipeline()

X_train, X_test, y_train, y_test = train_test_split(
    data.X, data.Y,
    test_size=0.2,
    random_state=42,
    stratify=data.Y
)

8 duplicates have been removed
6 seniorities found:
['junior', 'professional', 'senior', 'lead', 'management', 'director']


#### Random Forest

In [ ]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestClassifier(random_state=42, n_jobs=-1, class_weight="balanced"))
])

param_dist = {
    "rf__n_estimators": [200, 300, 400, 500],
    "rf__max_depth": [None, 5, 10, 20, 40],
    "rf__min_samples_split": [2, 5, 10, 20],
    "rf__min_samples_leaf": [1, 2, 4, 8],
    "rf__max_features": ["sqrt", "log2", None, 0.3, 0.5, 0.8],
    "rf__bootstrap": [True, False],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=40,
    scoring="f1_macro",
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)

print("Best CV score:", search.best_score_)
print("Best params:\n", search.best_params_)

best_model = search.best_estimator_

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END rf__bootstrap=False, rf__max_depth=40, rf__max_features=0.8, rf__min_samples_leaf=8, rf__min_samples_split=20, rf__n_estimators=400; total time=   0.7s
[CV] END rf__bootstrap=False, rf__max_depth=40, rf__max_features=0.8, rf__min_samples_leaf=8, rf__min_samples_split=20, rf__n_estimators=400; total time=   0.8s
[CV] END rf__bootstrap=False, rf__max_depth=40, rf__max_features=0.8, rf__min_samples_leaf=8, rf__min_samples_split=20, rf__n_estimators=400; total time=   0.7s
[CV] END rf__bootstrap=True, rf__max_depth=5, rf__max_features=0.8, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=800; total time=   1.4s
[CV] END rf__bootstrap=False, rf__max_depth=40, rf__max_features=0.8, rf__min_samples_leaf=8, rf__min_samples_split=20, rf__n_estimators=400; total time=   0.6s
[CV] END rf__bootstrap=True, rf__max_depth=5, rf__max_features=0.8, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=

In [10]:
y_pred = best_model.predict(X_test)
print("\nTest accuracy:", accuracy_score(y_test, y_pred))
print("\nTest report:\n", classification_report(y_test, y_pred))


Test accuracy: 0.2717391304347826

Test report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.44      0.21      0.28        34
           2       0.31      0.62      0.42         8
           3       0.11      0.05      0.07        20
           4       0.33      0.50      0.40        24
           5       0.00      0.00      0.00         4

    accuracy                           0.27        92
   macro avg       0.20      0.23      0.19        92
weighted avg       0.30      0.27      0.26        92



#### Decision Tree

In [11]:
pipe_dt = Pipeline([
    ("scaler", StandardScaler()),  # not required, but fine for consistency
    ("dt", DecisionTreeClassifier(random_state=42, class_weight="balanced"))
])

param_dist_dt = {
    "dt__max_depth": [None, 3, 5, 8, 12, 20, 40],
    "dt__min_samples_split": [2, 5, 10, 20, 50],
    "dt__min_samples_leaf": [1, 2, 4, 8, 16],
    "dt__max_features": ["sqrt", "log2", None, 0.3, 0.5, 0.8],
    "dt__criterion": ["gini", "entropy", "log_loss"],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search_dt = RandomizedSearchCV(
    estimator=pipe_dt,
    param_distributions=param_dist_dt,
    n_iter=40,
    scoring="f1_macro",
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search_dt.fit(X_train, y_train)

print("DT Best CV score:", search_dt.best_score_)
print("DT Best params:\n", search_dt.best_params_)

best_dt = search_dt.best_estimator_

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END dt__criterion=log_loss, dt__max_depth=20, dt__max_features=0.5, dt__min_samples_leaf=8, dt__min_samples_split=2; total time=   0.0s
[CV] END dt__criterion=log_loss, dt__max_depth=20, dt__max_features=0.5, dt__min_samples_leaf=8, dt__min_samples_split=2; total time=   0.0s
[CV] END dt__criterion=log_loss, dt__max_depth=20, dt__max_features=0.5, dt__min_samples_leaf=8, dt__min_samples_split=2; total time=   0.0s
[CV] END dt__criterion=log_loss, dt__max_depth=20, dt__max_features=0.5, dt__min_samples_leaf=8, dt__min_samples_split=2; total time=   0.0s
[CV] END dt__criterion=gini, dt__max_depth=40, dt__max_features=None, dt__min_samples_leaf=8, dt__min_samples_split=50; total time=   0.0s
[CV] END dt__criterion=log_loss, dt__max_depth=20, dt__max_features=0.5, dt__min_samples_leaf=8, dt__min_samples_split=2; total time=   0.0s
[CV] END dt__criterion=gini, dt__max_depth=40, dt__max_features=None, dt__min_samples_leaf=8, 

In [12]:
y_pred = best_dt.predict(X_test)
print("\nDT Test accuracy:", accuracy_score(y_test, y_pred))
print("\nDT Test report:\n", classification_report(y_test, y_pred))


DT Test accuracy: 0.2608695652173913

DT Test report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.40      0.29      0.34        34
           2       0.08      0.12      0.10         8
           3       0.27      0.30      0.29        20
           4       0.30      0.29      0.30        24
           5       0.00      0.00      0.00         4

    accuracy                           0.26        92
   macro avg       0.18      0.17      0.17        92
weighted avg       0.29      0.26      0.27        92



#### XGBBoost

In [4]:
pipe_xgb = Pipeline([
    ("scaler", StandardScaler()),  # not required for trees, but fine
    ("xgb", XGBClassifier(
        objective="multi:softprob",
        eval_metric="mlogloss",
        random_state=42,
        n_jobs=-1
    ))
])

param_dist_xgb = {
    "xgb__n_estimators": [200, 400, 800, 1200],
    "xgb__max_depth": [3, 4, 5, 6, 8, 10],
    "xgb__learning_rate": [0.01, 0.03, 0.05, 0.1, 0.2],
    "xgb__subsample": [0.6, 0.8, 1.0],
    "xgb__colsample_bytree": [0.6, 0.8, 1.0],
    "xgb__min_child_weight": [1, 3, 5, 10],
    "xgb__gamma": [0, 0.1, 0.3, 1.0],
    "xgb__reg_lambda": [0.5, 1.0, 2.0, 5.0],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search_xgb = RandomizedSearchCV(
    estimator=pipe_xgb,
    param_distributions=param_dist_xgb,
    n_iter=40,
    scoring="f1_macro",
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search_xgb.fit(X_train, y_train)

print("XGB Best CV score:", search_xgb.best_score_)
print("XGB Best params:\n", search_xgb.best_params_)

best_xgb = search_xgb.best_estimator_

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END xgb__colsample_bytree=0.6, xgb__gamma=0.3, xgb__learning_rate=0.1, xgb__max_depth=8, xgb__min_child_weight=3, xgb__n_estimators=200, xgb__reg_lambda=1.0, xgb__subsample=0.6; total time=   0.1s
[CV] END xgb__colsample_bytree=0.6, xgb__gamma=0.3, xgb__learning_rate=0.1, xgb__max_depth=8, xgb__min_child_weight=3, xgb__n_estimators=200, xgb__reg_lambda=1.0, xgb__subsample=0.6; total time=   0.2s
[CV] END xgb__colsample_bytree=0.6, xgb__gamma=0.3, xgb__learning_rate=0.1, xgb__max_depth=8, xgb__min_child_weight=3, xgb__n_estimators=200, xgb__reg_lambda=1.0, xgb__subsample=0.6; total time=   0.1s
[CV] END xgb__colsample_bytree=0.6, xgb__gamma=0.3, xgb__learning_rate=0.1, xgb__max_depth=8, xgb__min_child_weight=3, xgb__n_estimators=200, xgb__reg_lambda=1.0, xgb__subsample=0.6; total time=   0.1s
[CV] END xgb__colsample_bytree=0.6, xgb__gamma=0.3, xgb__learning_rate=0.1, xgb__max_depth=8, xgb__min_child_weight=3, xgb__n_esti

In [5]:
y_pred = best_xgb.predict(X_test)
print("\nXGB Test accuracy:", accuracy_score(y_test, y_pred))
print("\nXGB Test report:\n", classification_report(y_test, y_pred))


XGB Test accuracy: 0.2826086956521739

XGB Test report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.41      0.47      0.44        34
           2       0.00      0.00      0.00         8
           3       0.14      0.10      0.12        20
           4       0.26      0.33      0.29        24
           5       0.00      0.00      0.00         4

    accuracy                           0.28        92
   macro avg       0.14      0.15      0.14        92
weighted avg       0.25      0.28      0.26        92



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

#### Support Vector Machine

In [13]:
pipe_svm = Pipeline([
    ("scaler", StandardScaler()),          # REQUIRED for SVM
    ("svm", SVC(class_weight="balanced"))
])

param_dist_svm = {
    "svm__C": [0.1, 0.3, 1, 3, 10, 30, 100],
    "svm__kernel": ["rbf", "linear", "poly"],
    "svm__gamma": ["scale", "auto", 0.01, 0.03, 0.1, 0.3, 1.0],
    "svm__degree": [2, 3, 4],              # only for poly
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search_svm = RandomizedSearchCV(
    estimator=pipe_svm,
    param_distributions=param_dist_svm,
    n_iter=30,                 # SVM is slow – 30 is usually enough
    scoring="f1_macro",
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search_svm.fit(X_train, y_train)

print("SVM Best CV score:", search_svm.best_score_)
print("SVM Best params:\n", search_svm.best_params_)

best_svm = search_svm.best_estimator_

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END svm__C=0.3, svm__degree=2, svm__gamma=0.3, svm__kernel=rbf; total time=   0.0s
[CV] END svm__C=0.3, svm__degree=2, svm__gamma=0.3, svm__kernel=rbf; total time=   0.0s
[CV] END svm__C=0.3, svm__degree=2, svm__gamma=0.3, svm__kernel=rbf; total time=   0.0s
[CV] END svm__C=0.3, svm__degree=2, svm__gamma=0.3, svm__kernel=rbf; total time=   0.0s
[CV] END svm__C=0.3, svm__degree=2, svm__gamma=0.3, svm__kernel=rbf; total time=   0.0s
[CV] END svm__C=1, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=1, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=1, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=1, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=1, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=10, svm__degree=3, svm__gamma=0.01, svm__k

In [14]:
y_pred = best_svm.predict(X_test)
print("\nSVM Test accuracy:", accuracy_score(y_test, y_pred))
print("\nSVM Test report:\n", classification_report(y_test, y_pred))


SVM Test accuracy: 0.22826086956521738

SVM Test report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.40      0.29      0.34        34
           2       0.12      0.25      0.16         8
           3       0.13      0.10      0.11        20
           4       0.32      0.29      0.30        24
           5       0.00      0.00      0.00         4

    accuracy                           0.23        92
   macro avg       0.16      0.16      0.15        92
weighted avg       0.27      0.23      0.24        92

